<a href="https://colab.research.google.com/github/noaakwey/geobotany/blob/main/gee_cluster_tatarstan_GLAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee, eemont
# Initialize the library.
ee.Authenticate()
ee.Initialize(project='ee-landeco')

In [ ]:
#L89_stat_to_Assets
year = 2018
roi = ee.FeatureCollection("projects/ee-landeco/assets/tatarstan_grid")

l8 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
      .filterBounds(roi)
      .filter(ee.Filter.calendarRange(year,year+4,'year'))
      .filter(ee.Filter.calendarRange(4,11,'month'))
      .preprocess()
      .spectralIndices('vegetation')
      )
l9 = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
      .filterBounds(roi)
      .filter(ee.Filter.calendarRange(year,year+4,'year'))
      .filter(ee.Filter.calendarRange(4,11,'month'))
      .preprocess()
      .spectralIndices('vegetation')
      )
l89 = l8.merge(l9).sort("system:time_start")
l89_inx = l89.select(['BWDRVI', 'CIG', 'CVI', 'DSI', 'DSWI1', 'DVI', 'DVIplus', 'EVI2', 'FCVI', 'GARI', 'GBNDVI', 'GCC', 'GDVI', 'GEMI', 'GLI',
                      'GNDVI', 'GOSAVI', 'GRNDVI', 'GRVI', 'GSAVI', 'GVMI', 'IAVI', 'IKAW', 'IPVI', 'MCARI2', 'MGRVI', 'MNDVI',
                      'MNLI', 'MRBVI', 'MSAVI', 'MSI', 'MSR', 'MTVI2', 'NDDI', 'NDGI', 'NDII', 'NDMI', 'NDPI', 'NDVI', 'NDYI', 'NGRDI',
                      'NIRv', 'NIRvH2', 'NLI', 'NMDI', 'NRFIg', 'NRFIr', 'NormG', 'NormNIR', 'NormR', 'OCVI', 'OSAVI', 'RCC', 'RDVI', 'RGBVI', 'RGRI',
                      'RI', 'SARVI', 'SAVI', 'SEVI', 'SI', 'SIPI', 'SR', 'TDVI', 'TGI', 'TSAVI', 'TVI', 'TriVI', 'VARI', 'VIG', 'WDRVI', 'WDVI'])

l89_select = [l89_inx.reduce(ee.Reducer.median()).multiply(100),
              l89_inx.reduce(ee.Reducer.mean()).multiply(100),
              l89_inx.reduce(ee.Reducer.max()).multiply(100),
              l89_inx.reduce(ee.Reducer.stdDev()).multiply(100)]
l89_comp = ee.Image.cat([l89_select])

# Load the MASK dataset
mask_dataset = ee.Image("projects/ee-landeco/assets/GLAD_RT_MASK")
mask = mask_dataset.eq(1)
l89_Vi_mask = l89_comp.updateMask(mask).clip(roi).toShort()

In [ ]:
# Загрузка обучающих данных
training_data = ee.Image("projects/ee-landeco/assets/ForestMapRT_Braun-Blanquet").select('b1')

# Объединение данных
labeled_data = l89_Vi_mask.addBands(training_data)

# Создание сбалансированной выборки для обучения
training = labeled_data.stratifiedSample(
    numPoints=5000,  # Количество точек на класс
    classBand='b1',  # Указываем, что классы находятся в поле 'b1'
    region=roi,
    tileScale=16,
    scale=30,
    seed=0
)

In [ ]:
# Экспорт на Google Drive
task = ee.batch.Export.table.toDrive(**{
    'collection': training,
    'description': 'training',
    'fileFormat': 'CSV',
    'folder': 'Carbon'  # Укажите имя папки на Google Drive
})
task.start()

In [ ]:
# Загрузка обучающих данных
training_data = ee.Image("projects/ee-landeco/assets/ForestMapRT_Braun-Blanquet").select('b1')

# Объединение данных
labeled_data = l89_Vi_mask.addBands(training_data)

# Создание сбалансированной выборки для обучения
training = labeled_data.stratifiedSample(
    numPoints=500,  # Количество точек на класс
    classBand='b1',  # Указываем, что классы находятся в поле 'b1'
    region=roi,
    scale=30,
    seed=0
)

# Обучение модели
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50).train(
    features=training,
    classProperty='b1',
    inputProperties=l89_Vi_mask.bandNames()
)

# Классификация
#classified_image = l89_Vi_mask.classify(classifier)

# Получение Feature Importance
importances = classifier.explain().get('importance')
print(importances)

In [ ]:
# Make the training dataset.
training = l89_Vi_mask.sample(**{
    'region': roi.geometry(),
    'scale': 30,
    'numPixels': 20000,
    'tileScale': 16
})

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaXMeans(**{
    'minClusters': 36,
    'maxClusters': 50,
}).train(training)

# Apply clusterer model to each row_id
def apply_clusterer(feature):
    row_id = feature.get('row_id')
    roi_clip = roi.filter(ee.Filter.eq('row_id', row_id))
    result = l89_Vi_mask.clip(roi_clip.geometry()).cluster(clusterer)
    return result.set('row_id', row_id)

result_images = roi.map(apply_clusterer)

# Convert the image collection to a list of images
result_images_list = result_images.toList(result_images.size())

# Create mosaic of clustered images
mosaic_image = ee.ImageCollection.fromImages(result_images_list).mosaic()

# Define task configuration
task_config = {
    'image': mosaic_image.clip(roi.geometry()),
    'folder': "carbon",
    'description': 'forest_cluster_tatarstan_wekaXMeans35_all_layer_GLAD',
    'fileFormat': 'GeoTIFF',
    'crs': 'EPSG:32639',
    'scale': 30,
    'region': roi.geometry(),
    'maxPixels': 1e12
}

# Export the mosaic image to Google Drive
task = ee.batch.Export.image.toDrive(**task_config)
task.start()

АНАЛИЗ ПОЛУЧЕННЫХ РЕЗУЛЬТАОВ

In [ ]:
forest_community_map = ee.Image("users/ortho/ForestMapRT_Braun-Blanquet")
points = forest_community_map.stratifiedSample(**{
  'numPoints': 1000,
  'classBand': 'b1',  # Change this to the actual name of your classification band
  'region': forest_community_map.geometry(),
  'scale': 30,
  'geometries': True
})
vegetation_metrics = l89_Vi_mask.sampleRegions(**{
  'collection': points,
  'properties': ['class'],
  'scale': 30
})

# Список классов
classes = [1, 2, 3, 4, 5, 6, 7]

# Список вегетационных индексов
vegetation_indices = ['BWDRVI', 'CIG', 'CVI', 'DSI', 'DSWI1', 'DVI', 'DVIplus', 'EVI2', 'FCVI', 'GARI', 'GBNDVI', 'GCC', 'GDVI', 'GEMI', 'GLI',
                      'GNDVI', 'GOSAVI', 'GRNDVI', 'GRVI', 'GSAVI', 'GVMI', 'IAVI', 'IKAW', 'IPVI', 'MCARI2', 'MGRVI', 'MNDVI',
                      'MNLI', 'MRBVI', 'MSAVI', 'MSI', 'MSR', 'MTVI2', 'NDDI', 'NDGI', 'NDII', 'NDMI', 'NDPI', 'NDVI', 'NDYI', 'NGRDI',
                      'NIRv', 'NIRvH2', 'NLI', 'NMDI', 'NRFIg', 'NRFIr', 'NormG', 'NormNIR', 'NormR', 'OCVI', 'OSAVI', 'RCC', 'RDVI', 'RGBVI', 'RGRI',
                      'RI', 'SARVI', 'SAVI', 'SEVI', 'SI', 'SIPI', 'SR', 'TDVI', 'TGI', 'TSAVI', 'TVI', 'TriVI', 'VARI', 'VIG', 'WDRVI', 'WDVI']

# Преобразование точек в растр
points_image = points.reduceToImage(properties=['class'], reducer=ee.Reducer.first())

# Расчет статистики для каждого класса и каждого вегетационного индекса
stats = []
for class_id in classes:
    for index in vegetation_indices:
        # Создание маски для текущего класса
        class_mask = points_image.eq(class_id)

        # Применение маски к изображению с вегетационными индексами
        class_image = l89_Vi_mask.updateMask(class_mask)

        # Расчет статистики для текущего индекса
        reducers = ee.Reducer.mean().combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True).combine(
            reducer2=ee.Reducer.median(), sharedInputs=True).combine(reducer2=ee.Reducer.mode(), sharedInputs=True)
        reduced = class_image.reduceRegion(
            reducer=reducers,
            geometry=roi.geometry(),
            scale=30,
            tileScale=16,
            maxPixels=1e12,  # Увеличьте значение maxPixels
            bestEffort=True  # Добавьте этот параметр для автоматического выбора масштаба
            )

        # Добавление результатов в список
        stats.append(ee.Feature(None, reduced.set('class', class_id).set('index', index)))

# Создание FeatureCollection из результатов
stats_feature_collection = ee.FeatureCollection(stats)

# Экспорт на Google Drive
task = ee.batch.Export.table.toDrive(**{
    'collection': stats_feature_collection,
    'description': 'vegetation_metrics_stats',
    'fileFormat': 'CSV',
    'folder': 'Carbon'  # Укажите имя папки на Google Drive
})
task.start()

In [ ]:
forest_community_map = ee.Image("users/ortho/ForestMapRT_Braun-Blanquet")
points = forest_community_map.stratifiedSample(**{
  'numPoints': 1000,
  'classBand': 'b1',  # Change this to the actual name of your classification band
  'region': forest_community_map.geometry(),
  'scale': 30,
  'geometries': True
})
# Список классов
classes = [1, 2, 3, 4, 5, 6, 7]

# Список вегетационных индексов
vegetation_indices = ['BWDRVI', 'CIG', 'CVI', 'DSI', 'DSWI1', 'DVI', 'DVIplus', 'EVI2', 'FCVI', 'GARI', 'GBNDVI', 'GCC', 'GDVI', 'GEMI', 'GLI',
                      'GNDVI', 'GOSAVI', 'GRNDVI', 'GRVI', 'GSAVI', 'GVMI', 'IAVI', 'IKAW', 'IPVI', 'MCARI2', 'MGRVI', 'MNDVI',
                      'MNLI', 'MRBVI', 'MSAVI', 'MSI', 'MSR', 'MTVI2', 'NDDI', 'NDGI', 'NDII', 'NDMI', 'NDPI', 'NDVI', 'NDYI', 'NGRDI',
                      'NIRv', 'NIRvH2', 'NLI', 'NMDI', 'NRFIg', 'NRFIr', 'NormG', 'NormNIR', 'NormR', 'OCVI', 'OSAVI', 'RCC', 'RDVI', 'RGBVI', 'RGRI',
                      'RI', 'SARVI', 'SAVI', 'SEVI', 'SI', 'SIPI', 'SR', 'TDVI', 'TGI', 'TSAVI', 'TVI', 'TriVI', 'VARI', 'VIG', 'WDRVI', 'WDVI']

# Экспорт значений для каждого класса
for class_id in classes:
    # Фильтрация точек по текущему классу
    class_points = points.filter(ee.Filter.eq('b1', class_id))

    # Выборка значений вегетационных индексов в точках
    sampled_data = l89_Vi_mask.sampleRegions(**{
        'collection': class_points,
        'properties': ['b1'],
        'scale': 30,
        'geometries': True,
        'tileScale':16
    })

    # Экспорт данных на Google Drive
    task = ee.batch.Export.table.toDrive(**{
        'collection': sampled_data,
        'description': f'vegetation_metrics_class_{class_id}',
        'fileFormat': 'CSV',
        'folder': 'Carbon'  # Укажите имя папки на Google Drive
    })
    task.start()
    print(f'Task for class {class_id} started.')

Task for class 1 started.
Task for class 2 started.
Task for class 3 started.
Task for class 4 started.
Task for class 5 started.
Task for class 6 started.
Task for class 7 started.


In [ ]:
# Экспорт на Google Drive
task = ee.batch.Export.table.toDrive(**{
    'collection': points,
    'description': 'points',
    'fileFormat': 'CSV',
    'folder': 'Carbon'  # Укажите имя папки на Google Drive
})
task.start()